# Tools

A model's capabilities is extended with tools. Tools can be built into the API or accessed through functions and Model Context Protocol (MCP) services.

Tools include:

- Image generation.
- Web search.
- Function calling.
- Remote MCP Servers.
- File search.
- Code interpreter.
- Computer use.

# Function Calling

In this notebook we use function calling to illustrate tools and their usage. 

A useful resource is OpenAI's [Function Calling documentation for the responses API](https://platform.openai.com/docs/guides/function-calling?api-mode=responses).

In [ ]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [ ]:
from openai import OpenAI
import json
import os

client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

# Tool Calling Flow

The idea behind using tools is that a model will request to use a tool. When a model examines a prompt, it may determine that to follow the instruction it should use a tool. It then generates a special type of output, which we will catch and call a specific tool. 

The logic flows as follows:


1. Make a request to the model with tools it could call
2. Receive a tool call from the model
3. Execute code on the application side with input from the tool call
4. Make a second request to the model with the tool output
5. Receive a final response from the model (or more tool calls)


<img src="./img/05_function-calling-diagram-steps.png" height=700>

# Make a request to the model with tools it could call

The tool definition includes the function name, its parameters, and additional metadata. 

In [ ]:
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get horoscope for a given zodiac sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "zodiac_sign": {
                    "type": "string",
                    "description": "Zodiac sign e.g. Aries, Taurus",
                }
            },
            "required": ["zodiac_sign"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

def get_horoscope(zodiac_sign: str) -> str:
    # Dummy implementation for illustration
    horoscope = f"{zodiac_sign}: Today is a great day for new beginnings."
    return horoscope



We will need to retain some "memory" of the conversation. In this case, we will retain the message history and send it as context in our interaction with the model

In [ ]:
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

We prompt the model with the initial input.

In [ ]:
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Execute code on the application side with input from the tool call


Examine the response output. Notice that we find a "reasoning item" and a "function tool call".

In [ ]:
response.output

The function call item indicates that the model is requesting to run the function call: `get_horoscope(zodiac_sign='Aquarius')`.

In [ ]:
response.output[1].model_dump()

In the code below:

+ Execute the function logic for get_horoscope.
+ Add the function output to the input_list.

In [ ]:
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # Execute the function logic for get_horoscope
            horoscope = get_horoscope(**json.loads(item.arguments))
            
            # Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

# Make a second request to the model with the tool output


Examine the final input_list which is the context of model call.

In [ ]:
print(input_list)

In [ ]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)


# Receive a final response from the model (or more tool calls)

In [ ]:
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

## 💡 Frequently Asked Questions (FAQ)

**Q: Does the LLM run the code?**
A: No. The LLM generates *text* that looks like a function call (e.g., `get_weather(city="London")`). Your Python code (the runtime) sees this, executes the actual function, and sends the *result* back to the LLM.

**Q: What is a Tool Binding?**
A: It's the process of defining your function schemas (usually as JSON Schema) and passing them to the model API so it knows what tools are available.

**Q: Can I give it dangerous tools?**
A: Yes, and you shouldn't! If you give an LLM a tool `delete_database()`, it might use it. Always sandbox tool execution.
